In [88]:
import pyodbc as dbc
import pandas as pd
import sqlalchemy as sa
import os

DB1_NAME = 'opsmdd_1.mdb'
DB2_NAME = 'opsmdd_2.mdb'

In [51]:
def get_driver() -> str:
    driver_names = [x for x in dbc.drivers() if "*.mdb" in x]
    if driver_names:
        return driver_names[0]

In [99]:
# Function to connect to an Access Database (.mdb)
def connect_to_mdb(db_path:str) -> dbc.Connection:
    conn_str = r'DRIVER={}; DBQ={};'.format(get_driver(), db_path)
    connection_url = sa.engine.URL.create(
        "access+pyodbc",
        query={"odbc_connect": conn_str}
    )
    return sa.create_engine(connection_url)

In [63]:
# Function to read a table into a pandas DataFrame
# def read_table_old(connection:dbc.Connection, table_name:list) -> pd.DataFrame:
#     query = f"SELECT * FROM [{table_name}]"
#     return pd.read_sql(query, connection)

In [68]:
# Function to read a table into a pandas DataFrame
def read_table(connection:dbc.Connection, table_name:list) -> pd.DataFrame:
    return pd.read_sql(f"SELECT * FROM [{table_name}]", connection)
    # return pd.read_sql(query, connection)

In [86]:
# Get the current working directory
current_dir = os.getcwd()
# Build full paths for the database files
db1_path = os.path.join(current_dir, DB1_NAME)
db2_path = os.path.join(current_dir, DB2_NAME)

In [100]:
engine = connect_to_mdb(db1_path)

In [106]:
access_df = pd.read_sql('vars', engine)

In [110]:
type(engine)

sqlalchemy.engine.base.Engine

In [111]:
engine.dispose()